# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [1]:
import pandas as pd
import numpy as np

In [24]:
# Загрузка и подготовка данных
data = pd.read_csv("./data/Advertising.csv", index_col=0)

# Выведем данные по таблице
display(data.head())
print(data.describe(),data.info())


,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB
               TV       radio   newspaper       sales
count  200.000000  200.000000  200.000000  200.000000
mean   147.042500   23.264000   30.554000   14.022500
std     85.854236   14.846809   21.778621    5.217457
min      0.700000    0.000000    0.300000    1.600000
25%     74.375000    9.975000   12.750000   10.375000
50%    149.750000   22.900000   25.750000   12.900000
75%    218.825000   36.525000   45.100000   17.400000
max    296.400000   49.600000  114.000000   27.000000 None


----

**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [25]:
# Отпринтуем пропуски
print(data.isnull().sum())

TV           0
radio        0
newspaper    0
sales        0
dtype: int64


> Пропусков в данных нет, но если посмотреть на статистики на прошлом этапе, можно отметить нулевые значения в столбце радио, проверим их количество и удалим, если они не имеют большого веса в контексте выборки (в этом случае, можно будет предположить, что нулевые значения не являются пропусками, а скорее допустимой характеристикой переменной)

In [26]:
# Отфильтруем по минимальному-нулевому значению
data[data["radio"] == data["radio"].min()]

,TV,radio,newspaper,sales
128,80.2,0.0,9.2,8.8


> Как видно нулевое значение наличествует только в 1 строке, что можно рассмотреть как пропуск, удалим его

In [27]:
# Вырежем через фильтрацию
data = data[data["radio"] != data["radio"].min()]

# Посмотрим на полученный результат
display(data.head())
print(data.describe(),data.info())

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 1 to 200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         199 non-null    float64
 1   radio      199 non-null    float64
 2   newspaper  199 non-null    float64
 3   sales      199 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB
               TV       radio   newspaper       sales
count  199.000000  199.000000  199.000000  199.000000
mean   147.378392   23.380905   30.661307   14.048744
std     85.938922   14.791683   21.780479    5.217365
min      0.700000    0.300000    0.300000    1.600000
25%     74.050000   10.050000   12.850000   10.400000
50%    149.800000   23.300000   25.900000   12.900000
75%    219.150000   36.550000   45.100000   17.400000
max    296.400000   49.600000  114.000000   27.000000 None


> Нулевые и пропущенные значения отсутсвуют. Задача выполнена.
___

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [28]:
# ваш код
X = np.array(data.drop(columns=["sales"], axis=1))
y = np.array(data["sales"])


## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [29]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [30]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм:

<a href="https://ibb.co/Th3BQFn"><img src="https://i.ibb.co/DK2DBS6/zascas.jpg" alt="zascas" border="0"></a>

Примечание: 1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [31]:
# Исправленный код алгоритма, приведенного в задании
w = np.zeros(X.shape[1])
iterations = 1000

for iteration in range(iterations):
    r = y - np.dot(X, w)  # Вычисляем остатки

    for j in range(X.shape[1]):
        # Вычисляем остаток без j-го веса
        r_j_prev = r + X[:, j] * w[j]
        # Обновляем вес w[j]
        w[j] = np.dot(X[:, j], r_j_prev)
        # Обновляем остатки
        r = r_j_prev - X[:, j] * w[j]

print(w)

[ 40.6711837  110.28460324  73.93424192  -0.42416681]


Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [32]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)
model.fit(X, y)

print(model.coef_)

[ 40.6711837  110.28460324  73.93424192  -0.42416681]


Если вы все сделали верно, они должны практически совпасть!

> Результаты идентичны

----

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы еще не нормализовали). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [41]:
# Реинициализируем наше добро
X = np.array(data.drop(columns=["sales"], axis=1))
y = np.array(data["sales"])

In [42]:
# Нормируем
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

**Добавим единичный столбец**

In [43]:
X_norm = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [55]:
# Функция MSE
def mse_error(y_test, y_pred, round_n=3):
    mse = np.mean(np.square(y_test - y_pred))
    mse = np.round(mse, round_n)
    return mse

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [56]:
# Формируем прогноз на основе прокидывания среднего значения
y_mean = np.mean(y) # Вычисляем среднее значение
y_pred_mean = np.full_like(y, y_mean) # Создаем массив формы y, заполенный средним значением
mse_error = mse_error(y, y_pred_mean)
print(f"MSE на прогнозе средним: {mse_error}")

MSE на прогнозе средним: 27.084


**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [47]:
# Функция предсказания средним
def lin_pred(X, w):
    return np.dot(X, w)

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [64]:
def stoch_grad_step(X, y, w, train_ind, eta=0.01):
    # Количество объектов в выборке
    l = X.shape[0]
    
    # Получаем текущую обучающую строку
    x_i = X[train_ind]
    y_i = y[train_ind]

    # Предсказание для текущего объекта
    predicted = w[0] + np.dot(w[1:], x_i)  # Предсказание для текущего объекта
    error = predicted - y_i  # Ошибка для текущего объекта

    # Обновление весов
    w[0] = w[0] - eta * error  # Обновление свободного члена (w[0])
    
    for j in range(1, len(w)):  # Начинаем с 1, чтобы пропустить w[0]
        gradient = x_i[j - 1] * error  # Градиент по w_j
        w[j] = w[j] - (2 * eta / l) * gradient  # Обновление веса

    return w

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [16]:
def stochastic_gradient_descent(X, y, ):
    pass

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [17]:
# ваш код

**Выведите вектор весов, к которому сошелся метод.**

In [18]:
# ваш код

**Выведите среднеквадратичную ошибку на последней итерации.**

In [19]:
# ваш код